In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet


target_shape = (200, 200)

In [2]:
from PIL import Image
import PIL.ImageOps   
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

D:\myApps\anaconda\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# Creating some helper functions
def imshow(img, text=None):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
        
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [4]:

class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset,transform=None):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)

        #We need to approximately 50% of images to be in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #Look untill the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] == img1_tuple[1]:
                    break
        else:

            while True:
                #Look untill a different class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] != img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])

        img0 = img0.convert("L")
        img1 = img1.convert("L")

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [11]:
folder_dataset = datasets.ImageFolder(root="D:/machine learning/Applai/Face Recognision/archive/train")

In [12]:
# Resize the images and transform to tensors
transformation = transforms.Compose([transforms.Resize((100,100)),
                                     transforms.ToTensor()
                                    ])

# Initialize the network
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset,
                                        transform=transformation)

In [13]:
# Create a simple dataloader just for simple visualization
vis_dataloader = DataLoader(siamese_dataset,
                        shuffle=True)

In [14]:
# Extract one batch
example_batch = next(iter(vis_dataloader))

In [15]:
# Example batch is a list containing 2x8 images, indexes 0 and 1, an also the label
# If the label is 1, it means that it is not the same person, label is 0, same person in both images
concatenated = torch.cat((example_batch[0], example_batch[1]),0)

In [16]:
imshow(torchvision.utils.make_grid(concatenated))
print(example_batch[2].numpy().reshape(-1))

[0.]


In [17]:
#create the Siamese Neural Network
class SiameseNetwork(nn.Module):

    def __init__(self):
        super(SiameseNetwork, self).__init__()

        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11,stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            
            nn.Conv2d(96, 256, kernel_size=5, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(256, 384, kernel_size=3,stride=1),
            nn.ReLU(inplace=True)
        )

        # Setting up the Fully Connected Layers
        self.fc1 = nn.Sequential(
            nn.Linear(384, 1024),
            nn.ReLU(inplace=True),
            
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            
            nn.Linear(256,2)
        )
        
    def forward_once(self, x):
        # This function will be called for both images
        # It's output is used to determine the similiarity
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        # In this function we pass in both images and obtain both vectors
        # which are returned
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        return output1, output2

In [18]:
# Define the Contrastive Loss Function
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
      # Calculate the euclidian distance and calculate the contrastive loss
      euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

      loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


      return loss_contrastive

In [20]:
# Load the training dataset
train_dataloader = DataLoader(siamese_dataset,
                        shuffle=True)

In [21]:
net = SiameseNetwork().cuda()
criterion = ContrastiveLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.0005 )

In [ ]:
counter = []
loss_history = [] 
iteration_number= 0

# Iterate throught the epochs
for epoch in range(100):

    # Iterate over batches
    for i, (img0, img1, label) in enumerate(train_dataloader, 0):

        # Send the images and labels to CUDA
        img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()

        # Zero the gradients
        optimizer.zero_grad()

        # Pass in the two images into the network and obtain two outputs
        output1, output2 = net(img0, img1)

        # Pass the outputs of the networks and label into the loss function
        loss_contrastive = criterion(output1, output2, label)

        # Calculate the backpropagation
        loss_contrastive.backward()

        # Optimize
        optimizer.step()

        # Every 10 batches print out the loss
        if i % 10 == 0 :
            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
            iteration_number += 10

            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())

show_plot(counter, loss_history)

Epoch number 0
 Current loss 4.110613645025296e-06

Epoch number 0
 Current loss 3.9430480003356934

Epoch number 0
 Current loss 0.3986937999725342

Epoch number 0
 Current loss 1.4069349765777588

Epoch number 0
 Current loss 0.8454982042312622

Epoch number 0
 Current loss 0.07038399577140808

Epoch number 0
 Current loss 2.8252310752868652

Epoch number 0
 Current loss 0.47360700368881226

Epoch number 0
 Current loss 1.2906360626220703

Epoch number 0
 Current loss 3.8696537017822266

Epoch number 0
 Current loss 0.24449478089809418

Epoch number 0
 Current loss 0.026713112369179726

Epoch number 0
 Current loss 0.09541098028421402

Epoch number 0
 Current loss 0.9612776041030884

Epoch number 0
 Current loss 1.4563555717468262

Epoch number 0
 Current loss 0.08372559398412704

Epoch number 0
 Current loss 0.03569645434617996

Epoch number 0
 Current loss 0.16353388130664825

Epoch number 0
 Current loss 1.1238857507705688

Epoch number 0
 Current loss 3.974055528640747

Epoch num

Epoch number 0
 Current loss 1.6145209074020386

Epoch number 0
 Current loss 1.445973515510559

Epoch number 0
 Current loss 0.0

Epoch number 0
 Current loss 3.3071889877319336

Epoch number 0
 Current loss 0.07341733574867249

Epoch number 0
 Current loss 3.4543776512145996

Epoch number 0
 Current loss 1.7767976522445679

Epoch number 0
 Current loss 0.005563114304095507

Epoch number 0
 Current loss 0.02406235784292221

Epoch number 0
 Current loss 1.2415765523910522

Epoch number 0
 Current loss 0.003826719941571355

Epoch number 0
 Current loss 1.2978944778442383

Epoch number 0
 Current loss 0.030071889981627464

Epoch number 0
 Current loss 0.04925788938999176

Epoch number 0
 Current loss 2.676440715789795

Epoch number 0
 Current loss 2.429746627807617

Epoch number 0
 Current loss 0.021351920440793037

Epoch number 0
 Current loss 0.002466593636199832

Epoch number 0
 Current loss 1.323551893234253

Epoch number 0
 Current loss 2.821911096572876

Epoch number 0
 Current los

Epoch number 0
 Current loss 0.39754876494407654

Epoch number 0
 Current loss 3.496751546859741

Epoch number 0
 Current loss 0.01202615350484848

Epoch number 0
 Current loss 0.03523438051342964

Epoch number 0
 Current loss 3.8465967178344727

Epoch number 0
 Current loss 0.06670945882797241

Epoch number 0
 Current loss 0.7076032161712646

Epoch number 0
 Current loss 2.49652361869812

Epoch number 0
 Current loss 0.10866320133209229

Epoch number 0
 Current loss 13.858932495117188

Epoch number 0
 Current loss 1.292084813117981

Epoch number 0
 Current loss 0.8636921048164368

Epoch number 0
 Current loss 1.2470389604568481

Epoch number 0
 Current loss 1.1811585426330566

Epoch number 0
 Current loss 0.0030189582612365484

Epoch number 0
 Current loss 0.017357168719172478

Epoch number 0
 Current loss 2.8690319061279297

Epoch number 0
 Current loss 2.7280967235565186

Epoch number 0
 Current loss 0.021301008760929108

Epoch number 0
 Current loss 0.2673720419406891

Epoch number

Epoch number 0
 Current loss 1.9277547597885132

Epoch number 0
 Current loss 0.04257519915699959

Epoch number 0
 Current loss 0.1639064997434616

Epoch number 0
 Current loss 0.0908629298210144

Epoch number 0
 Current loss 1.8319687843322754

Epoch number 0
 Current loss 3.2712886333465576

Epoch number 0
 Current loss 2.1883533000946045

Epoch number 0
 Current loss 0.27592289447784424

Epoch number 0
 Current loss 0.001019549323245883

Epoch number 0
 Current loss 0.32199832797050476

Epoch number 0
 Current loss 0.17625099420547485

Epoch number 0
 Current loss 0.02058611810207367

Epoch number 0
 Current loss 0.2210322469472885

Epoch number 0
 Current loss 3.1987197399139404

Epoch number 0
 Current loss 2.1819424629211426

Epoch number 0
 Current loss 2.940810203552246

Epoch number 0
 Current loss 0.02318931743502617

Epoch number 0
 Current loss 0.006778848823159933

Epoch number 0
 Current loss 3.9424753189086914

Epoch number 0
 Current loss 0.0083418944850564

Epoch numbe

Epoch number 0
 Current loss 1.1518365144729614

Epoch number 0
 Current loss 2.1860616207122803

Epoch number 0
 Current loss 1.2034008502960205

Epoch number 0
 Current loss 0.11098848283290863

Epoch number 0
 Current loss 2.0888290405273438

Epoch number 0
 Current loss 2.1377224922180176

Epoch number 0
 Current loss 0.6968285441398621

Epoch number 0
 Current loss 1.1847712993621826

Epoch number 0
 Current loss 0.10318085551261902

Epoch number 0
 Current loss 1.9698723554611206

Epoch number 0
 Current loss 3.955148696899414

Epoch number 0
 Current loss 0.2839029133319855

Epoch number 0
 Current loss 0.11943038552999496

Epoch number 0
 Current loss 2.1923418045043945

Epoch number 0
 Current loss 1.8856407403945923

Epoch number 0
 Current loss 0.7500121593475342

Epoch number 0
 Current loss 0.03255700320005417

Epoch number 0
 Current loss 1.7209793329238892

Epoch number 0
 Current loss 0.12821966409683228

Epoch number 0
 Current loss 1.2293508052825928

Epoch number 0
 

...

Epoch number 6
 Current loss 0.8421517014503479

Epoch number 6
 Current loss 0.20639532804489136

Epoch number 6
 Current loss 0.07420878857374191

Epoch number 6
 Current loss 0.26887181401252747

Epoch number 6
 Current loss 0.0

Epoch number 6
 Current loss 1.9289406538009644

Epoch number 6
 Current loss 0.7387303113937378

Epoch number 6
 Current loss 0.8184696435928345

Epoch number 6
 Current loss 2.13722825050354

Epoch number 6
 Current loss 0.36108502745628357

Epoch number 6
 Current loss 0.03541736677289009

Epoch number 6
 Current loss 1.7451860904693604

Epoch number 6
 Current loss 0.6436886191368103

Epoch number 6
 Current loss 0.039861708879470825

Epoch number 6
 Current loss 0.04734526574611664

Epoch number 6
 Current loss 0.14146287739276886

Epoch number 6
 Current loss 0.718210756778717

Epoch number 6
 Current loss 0.0

Epoch number 6
 Current loss 0.30923184752464294

Epoch number 6
 Current loss 0.0

Epoch number 6
 Current loss 0.7996507287025452

Epoch num

Epoch number 7
 Current loss 2.321462869644165

Epoch number 7
 Current loss 0.2462082803249359

Epoch number 7
 Current loss 2.1968178749084473

Epoch number 7
 Current loss 3.4008467197418213

Epoch number 7
 Current loss 0.49405139684677124

Epoch number 7
 Current loss 3.5035886764526367

Epoch number 7
 Current loss 0.9843456745147705

Epoch number 7
 Current loss 0.8908841609954834

Epoch number 7
 Current loss 0.029149219393730164

Epoch number 7
 Current loss 2.6029655933380127

Epoch number 7
 Current loss 1.3759952783584595

Epoch number 7
 Current loss 0.6826315522193909

Epoch number 7
 Current loss 0.5911962389945984

Epoch number 7
 Current loss 0.024524889886379242

Epoch number 7
 Current loss 1.3760919570922852

Epoch number 7
 Current loss 0.024030424654483795

Epoch number 7
 Current loss 0.08193319290876389

Epoch number 7
 Current loss 0.6329788565635681

Epoch number 7
 Current loss 1.3113329410552979

Epoch number 7
 Current loss 0.0

Epoch number 7
 Current loss

Epoch number 7
 Current loss 0.01593196764588356

Epoch number 7
 Current loss 0.12932103872299194

Epoch number 7
 Current loss 2.2579073905944824

Epoch number 7
 Current loss 0.5542330741882324

Epoch number 7
 Current loss 1.0183043479919434

Epoch number 7
 Current loss 0.0

Epoch number 7
 Current loss 0.0

Epoch number 7
 Current loss 0.4780183732509613

Epoch number 7
 Current loss 0.5570207834243774

Epoch number 7
 Current loss 0.0

Epoch number 7
 Current loss 0.015190817415714264

Epoch number 7
 Current loss 0.36502155661582947

Epoch number 7
 Current loss 0.1579750031232834

Epoch number 7
 Current loss 1.4612829685211182

Epoch number 7
 Current loss 1.446186900138855

Epoch number 7
 Current loss 4.73575496673584

Epoch number 7
 Current loss 0.0

Epoch number 7
 Current loss 0.14365598559379578

Epoch number 7
 Current loss 1.0888844728469849

Epoch number 7
 Current loss 0.20413997769355774

Epoch number 7
 Current loss 0.26925426721572876

Epoch number 7
 Current lo

Epoch number 7
 Current loss 6.798795700073242

Epoch number 7
 Current loss 0.3251591920852661

Epoch number 7
 Current loss 0.6875463724136353

Epoch number 7
 Current loss 0.007454655133187771

Epoch number 7
 Current loss 0.4507608413696289

Epoch number 7
 Current loss 0.20311535894870758

Epoch number 7
 Current loss 0.6331494450569153

Epoch number 7
 Current loss 0.14001037180423737

Epoch number 7
 Current loss 0.0

Epoch number 7
 Current loss 0.2577188313007355

Epoch number 7
 Current loss 0.2613928020000458

Epoch number 7
 Current loss 0.0

Epoch number 7
 Current loss 0.8385271430015564

Epoch number 7
 Current loss 0.40633928775787354

Epoch number 7
 Current loss 2.093709707260132

Epoch number 7
 Current loss 0.16282372176647186

Epoch number 7
 Current loss 1.53476083278656

Epoch number 7
 Current loss 0.4622318148612976

Epoch number 7
 Current loss 0.613591730594635

Epoch number 7
 Current loss 1.353818416595459

Epoch number 7
 Current loss 0.0027057407423853874

Epoch number 7
 Current loss 0.0

Epoch number 7
 Current loss 2.1777782440185547

Epoch number 7
 Current loss 0.0

Epoch number 7
 Current loss 0.7025336027145386

Epoch number 7
 Current loss 0.7809756398200989

Epoch number 7
 Current loss 1.9384269714355469

Epoch number 7
 Current loss 0.14293521642684937

Epoch number 7
 Current loss 2.1658120155334473

Epoch number 7
 Current loss 1.311938762664795

Epoch number 7
 Current loss 1.792855143547058

Epoch number 7
 Current loss 0.05893215909600258

Epoch number 7
 Current loss 0.2605455815792084

Epoch number 7
 Current loss 0.05523207411170006

Epoch number 7
 Current loss 2.547773838043213

Epoch number 7
 Current loss 0.2702426016330719

Epoch number 7
 Current loss 0.18389861285686493

Epoch number 7
 Current loss 0.3055291175842285

Epoch number 7
 Current loss 0.8574655055999756

Epoch number 7
 Current loss 1.9357192516326904

Epoch number 7
 Current loss 1.0086134672164917

Epoch number 7
 Current loss 1.0275427103042603

